In [2]:
import gym
import tianshou as ts
import torch
import platoonenv

In [3]:
envName = "Platoon-v0"
# envName = "Platoon-v0"
env = gym.make(envName)

d:\Repos\School\Vehicular-Game-Theory\platoongame\platoonenv.py:40: UserWarning: WARN: pls ignore gym complaints about observation space
  gym.logger.warn("pls ignore gym complaints about observation space")
c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
train_envs = ts.env.DummyVectorEnv([lambda: gym.make(envName) for _ in range(10)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make(envName) for _ in range(100)])

In [5]:
from tianshou.utils.net.common import Net
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(state_shape, action_shape, hidden_sizes=(128,128,128,128))
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

In [6]:
print(state_shape, action_shape)

(120,) 11


In [7]:
policy = ts.policy.DQNPolicy(net, optim, discount_factor=0.9, estimation_step=3, target_update_freq=320)

In [8]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, train_envs, exploration_noise=True)

In [9]:
env.spec

EnvSpec(id='Platoon-v0', entry_point='platoonenv:PlatoonEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=None, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={}, namespace=None, name='Platoon', version=0)

In [10]:
threshold = env.spec.reward_threshold or 9
print(threshold)

9


In [12]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=20, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= threshold,
)
print(f"Finished training! Use {result['duration']}")

KeyboardInterrupt: 

In [ ]:
result

{'duration': '186.48s',
 'train_time/model': '86.97s',
 'test_step': 857772,
 'test_episode': 5400,
 'test_time': '77.86s',
 'test_speed': '11017.17 step/s',
 'best_reward': 500.0,
 'best_result': '500.00 ± 0.00',
 'train_step': 137170,
 'train_episode': 932,
 'train_time/collector': '21.65s',
 'train_speed': '1262.85 step/s'}

In [ ]:
policy.eval()
policy.set_eps(0.05)
collector = ts.data.Collector(policy, gym.make(envName, render_mode="human"), exploration_noise=False)
collector.collect(n_episode=10, render=1 / 60)

c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\tianshou\data\collector.py:68: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


KeyboardInterrupt: 